# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [2]:
with open("homeworkData.txt",encoding="utf-8") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [3]:
from langchain.text_splitter import CharacterTextSplitter

In [6]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 150,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [7]:
docs = text_splitter.create_documents([real_estate_sales])

In [8]:
docs[0]

Document(page_content='[客户问题]为什么要选择ROG的显卡？\n[销售回答]ROG显卡以其强大的性能和稳定性著称，拥有先进的散热技术和卓越的设计，适合高端游戏和图形处理需求。')

In [9]:
len(docs)

74

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings(

))

In [12]:
query = "卖的什么显卡"

In [13]:
answer_list = db.similarity_search(query)

In [14]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]ROG显卡的价格会随着新款发布而降价吗？
[销售回答]新款显卡发布可能会影响老款显卡的价格，但ROG显卡的品质和性能始终保持高水准，具体降价信息请关注我们的促销活动。

[客户问题]为什么要选择ROG的显卡？
[销售回答]ROG显卡以其强大的性能和稳定性著称，拥有先进的散热技术和卓越的设计，适合高端游戏和图形处理需求。

[客户问题]我需要一款适合视频编辑的显卡，ROG有推荐吗？
[销售回答]当然，ROG的Strix系列显卡具备强大的图形处理能力和大容量显存，非常适合视频编辑和渲染工作。

[客户问题]ROG显卡的价格为什么比其他品牌贵？
[销售回答]ROG显卡在品质、性能和售后服务上都有显著优势，使用高端组件和先进技术，确保最佳的使用体验和耐用性。



In [15]:
db.save_local("real_estates_sale_homework")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [74]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [75]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002878FFD5FF0>, search_kwargs={'k': 3})

In [76]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]ROG显卡的价格会随着新款发布而降价吗？
[销售回答]新款显卡发布可能会影响老款显卡的价格，但ROG显卡的品质和性能始终保持高水准，具体降价信息请关注我们的促销活动。

[客户问题]为什么要选择ROG的显卡？
[销售回答]ROG显卡以其强大的性能和稳定性著称，拥有先进的散热技术和卓越的设计，适合高端游戏和图形处理需求。

[客户问题]我需要一款适合视频编辑的显卡，ROG有推荐吗？
[销售回答]当然，ROG的Strix系列显卡具备强大的图形处理能力和大容量显存，非常适合视频编辑和渲染工作。



In [77]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的豪宅啊？")

In [79]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]ROG显卡是否适合搭建家庭影院系统？
[销售回答]当然，ROG显卡支持高分辨率和HDR显示，非常适合用于家庭影院系统，提供影院级的视听体验。

[客户问题]ROG显卡是否适合搭建多显卡系统？
[销售回答]是的，ROG显卡支持NVIDIA的SLI技术，您可以搭建多显卡系统，提升整体图形处理能力。

[客户问题]ROG显卡适合高分辨率游戏吗？
[销售回答]ROG显卡具备强大的图形处理能力，支持4K及更高分辨率的游戏，提供极致的画面质量和流畅的游戏体验。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [84]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [85]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

C:\czydev\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


### 提取向量数据库中的`销售回答`

In [82]:
docs = retriever.get_relevant_documents(query)

C:\czydev\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [83]:
docs[0].page_content

IndexError: list index out of range

In [22]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我担心楼下太吵。\n', '这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']

In [23]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [24]:
ans

'这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

#### 尝试各种问题

In [25]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [26]:
query = "我想离医院近点"

print(sales(query))

[]


/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain/schema/vectorstore.py:287: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [27]:
print(sales(query, 0.75))

['有的，距离我们小区不远就有几家大型综合医院。', '是的，附近有多家大型医院，医疗资源非常丰富。']


In [28]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: []

score:0.75 ans: []



/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain/schema/vectorstore.py:287: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.5 ans: ['我们有不同户型和付款方案，一定有适合您预算的。', '我们的房子位于黄金地段，升值潜力很大，转卖起来不会有问题。', '定金金额取决于您选择的房型和付款方式，我们可以详细为您解释。', '这个区域正在快速发展，未来的升值潜力非常大。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [29]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [31]:
qa_chain({"query": "你们小区有200万的房子吗？"})

{'query': '你们小区有200万的房子吗？',
 'result': '对不起，我无法回答这个问题，因为我是一个人工智能，没有实时的房地产信息。建议你直接联系房地产经纪人或者查阅相关房地产网站获取信息。'}

In [32]:
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵',
 'result': '这个小区特别注重居住体验，有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。所以，小区应该不会太吵。'}

In [33]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


## 加载 FAISS 向量数据库已有结果

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings())

In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [36]:
qa_chain({"query": "我想买别墅，你们有么"})

{'query': '我想买别墅，你们有么',
 'result': '对不起，我无法提供这样的服务。我是一个人工智能，我可以帮助回答问题，但我不能出售商品或房产。'}

In [37]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [38]:
qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅，因为我是一个人工智能，没有提供房地产服务的功能。'}

In [39]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [40]:
result = qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [41]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮你购买别墅。我是一个AI助手，我主要用来提供信息和回答问题。',
 'source_documents': []}

In [39]:
from langchain_openai import OpenAIEmbeddings
from langchain.chains import  ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
db = FAISS.load_local("real_estates_sale_homework", OpenAIEmbeddings(

    ),allow_dangerous_deserialization=True)
llm = ChatOpenAI(model_name="gpt-4", temperature=0,

                     )
    
global SALES_BOT
    # global MEMORY_BOT
memory = ConversationBufferMemory(max_token_limit=1000,memory_key="chat_history",return_messages=True, output_key="answer")

prompt_template ="你是一个优秀的显卡销售，你会介绍好自己的产品，会想办法把自己的产品推销出去，当顾客问到你不知道或不确定的问题，你会委婉的说出自己不清楚，并给到顾客解决方案和情绪价值。"
PROMPT = PromptTemplate(template=prompt_template)
chain_type_kwargs = {"prompt":PROMPT}
SALES_BOT =  ConversationalRetrievalChain.from_llm(llm,memory=memory,
                                           retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                     search_kwargs={"score_threshold": 0.8})
                                                  ,chain_type_kwargs=chain_type_kwargs)
    # 返回向量数据库的检索结果
SALES_BOT.return_source_documents = True

ValidationError: 1 validation error for ConversationalRetrievalChain
chain_type_kwargs
  extra fields not permitted (type=value_error.extra)

In [36]:
SALES_BOT({"question":"请介绍一下你自己"})

C:\czydev\miniconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores\base.py:745: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'question': '请介绍一下你自己',
 'chat_history': [HumanMessage(content='请介绍一下Rog显卡'),
  AIMessage(content='ROG显卡是一款以强大性能和稳定性著称的显卡。它具有先进的散热技术和卓越的设计，非常适合高端游戏和图形处理需求。在市场上，ROG显卡享有极高的声誉和广大用户的一致好评，被誉为性能强劲、品质优秀的显卡品牌。在各大评测机构的性能测试中，ROG显卡的表现优异，图形处理能力和稳定性均处于行业领先水平。'),
  HumanMessage(content='请介绍一下你自己'),
  AIMessage(content='当然可以。我是一个AI助手，专为用户提供各种信息查询、建议和解决问题。我的目的是为用户提供最准确、最有用的信息。')],
 'answer': '当然可以。我是一个AI助手，专为用户提供各种信息查询、建议和解决问题。我的目的是为用户提供最准确、最有用的信息。',
 'source_documents': []}